In [1]:
#
# Import Libraries
#

import pandas as pd
from dotmap import DotMap
import os

In [2]:
#
# Constants
#

modes = ['C18N', 'C18P', 'HILN', 'HILP']
f2i_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA\OriginalFiles\RBR_f2i.xlsx"

alid_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA\OriginalFiles\RBR_V1_Identifications.xlsx"

tp_path = r"S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA\WorkingFiles\Identifications\TP_results"

In [3]:
# Alessia manual identifications

alid = pd.read_excel(alid_path, sheet_name='Sheet3')

alid['Name'] = [i[0]+i[2:] for i in alid['Name']]

alid['Platform'] = [{
    'HILIC+':'HILP',
    'HILIC-':'HILN',
    'C18+':'C18P',
    'C18-':'C18N'
}[i]
for i in alid['Platform']]

alid = {i[0]:i[1] for i in alid.groupby('Platform')}

In [8]:
# Feature information (fid, rt, apex)

f2i = DotMap({
    i: pd.read_excel(f2i_path, sheet_name=i)
    for i in modes
})

In [9]:
# Add information from Aless
for i in modes:
    f2i[i] = pd.merge(
        f2i[i],
        alid[i].drop(['Apex m/z', 'RT [min]', 'Platform'], axis=1),
        on='Name',
        how='left'
    ).fillna('')

In [10]:
i = modes[0]

tp = {
    i: pd.read_csv(
        os.path.join(tp_path, i, [filename for filename in os.listdir(os.path.join(tp_path, i)) if 'TPFilter' in filename][0]),
        sep='\t', low_memory=False
    ).loc[:, ['FeatureInfo_Name','Name','Peptide','Halogenated','Plant','NaturalProduct','MDM','Drug','Food','Adduct','TP_Class_argmax','TPMetrics']]
    for i in modes
}

tp = {
    i: tp[i].fillna('')\
        .groupby('FeatureInfo_Name')\
            .agg(lambda x: ' | '.join([str(i) for i in list(x)]))\
                .reset_index()\
                    .rename(columns={'Name': 'TP_ID', 'Adduct':'TP_Adduct', 'FeatureInfo_Name':'fid'})
    for i in modes
}

In [11]:
for i in modes:
    f2i[i] = pd.merge(
        f2i[i],
        tp[i],
        on='fid',
        how='left'
    ).fillna('UNK')

In [12]:
f2i = pd.concat([
    f2i[i] for i in modes
])

In [22]:
# There are duplications generated by Bio_Class column
f2i = f2i[~f2i.fid.duplicated()]

In [23]:
f2i.to_csv('f2i.tsv', sep='\t', index=False)